## API

In [5]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [ ]:
url = "https://api.coindesk.com/v1/bpi/historical/close.json"

def request(currency="USD",sd=None,ed=None):
    """The currency must be in ISO 4217 format. The Start Date (sd) and the End date (ed) must be in the format YYYY-MM-DD, 
      if you dont write any argument you will get the last 31 days """     
   
    try: 
        if sd==None and ed==None:
            response = requests.get(url+ "?currency="+currency)
        elif sd!=None and ed != None:
            response = requests.get(url+ "?currency="+ currency+ "&start=" + sd + "&end="+ed)

        results = response.json()
        df = pd.DataFrame(results)
        df.reset_index(level=0, inplace=True)
        df2 = df[["index","bpi"]]
        df2.columns = [["Date","Bitcoin Price {}".format(currency)]]
        df2.reset_index(level=0)
        display(df2[:-2])

    except: 
        print("You must introduce both the sd and the ed of none of them. Verify that the currency exists")
    

#request("EUR","2017-01-01","2017-12-31").head()


In [ ]:
response = requests.get('https://api.coindesk.com/v1/bpi/historical/close.json')
results = response.json()
df = pd.DataFrame(results)
df.reset_index(level=0, inplace=True)
df2 = df[["index","bpi"]]
df2.columns = [["Date","Bitcoin Price (USD)"]]
df3 = df2[:30]
df3.head()
data = df3.to_csv('Output/API_Bitcoin.csv', index=False)

# Version 2 API

In [30]:
# Genero inputs para poder hacer la request a la API de coindesk con los parametros adecuados
list_months = {
"january":"31", "february":"28","march":"31","april":"30","may":"31","june":"30",
"july":"31","august":"31","september":"30","october":"31","november":"30","december":"31"
}
meses = list(list_months.keys())
while 1: 
    while 1:
        try:
            s_year = int(input("Please introduce the start year in format YYYY (from 2011 to 2018): "))
        except: 
            print("The format of the year is not valid, try again ...")
        else:
            if s_year not in range (2011,2019):
                print("The year is not between 2011 - 2018, tray again")
            if s_year in range(2011,2019):
                break

    while 1:
        try:
            e_year = int(input("Please introduce the end year in format YYYY (from 2011 to 2018): "))
        except: 
            print("The format of the year is not valid, try again ...")
        else:
            if e_year not in range (2011,2019):
                print("The year is not between 2011 - 2018, tray again")
            if e_year in range(2011,2019):
                break
                
    if (s_year > e_year):
        print("The end year must be preceded by the start year,try again ...")
        continue        
            
    while 1:
        try:
            s_month = input("Please introduce the start month (january,february,etc...): ").lower()
        except: 
            print("The format of the month is not valid, try again ...")
        else:
            if s_month in list_months.keys():
                break

    while 1:
        try:
            e_month = input("Please introduce the end month (january,february,etc...): ").lower()
        except: 
            print("The format of the month is not valid, try again ...")
        else:
            if e_month in list_months.keys():
                break
                
    if (s_year == e_year) and ( (meses.index(s_month)) > (meses.index(e_month))):
        print("The end month must be preceded by the start month, try again ...")
        continue      
    else: 
        break 

Please introduce the start year in format YYYY (from 2011 to 2018): 2011
Please introduce the end year in format YYYY (from 2011 to 2018): 2018
Please introduce the start month (january,february,etc...): january
Please introduce the end month (january,february,etc...): september


In [31]:
# Construyo el request para la api en funcion de los parametros recibidos
url_inic = "https://api.coindesk.com/v1/bpi/historical/close.json"
if s_month in meses[9:]:
    start = str(s_year) + "-" + str(meses.index(s_month)+1) + "-" + "01"
else: 
    start = str(s_year) + "-" + "0" + str(meses.index(s_month)+1) + "-" + "01"

if e_month in meses[9:]:
    ends = str(e_year) + "-"+ str(meses.index(e_month)+1) + "-" +str(list_months[e_month])
else:
    ends = str(e_year) + "-"+ "0" + str(meses.index(e_month)+1) + "-" +str(list_months[e_month])
    

# Envio el request a la API y recibo la respuesta 
response = requests.get(url_inic + "?start=" + start + "&end=" + ends)
results = response.json()

# Transformo la respuesta en un DataFrame y lo limpio para quedarme con las columnas que me interesa 
df_ = pd.DataFrame(results)
df = df_[["bpi"]]
df.columns = ["Bitcoin Price (USD)"]
df = df[:-2]
df.index = pd.to_datetime(df.index)
df
avg_year = df.groupby(df.index.year).mean()
avg_year.reset_index()
avg_year.columns = ["Avg yearly Bitcoin price (USD)"]
display (avg_year)

# Calculo el precio promedio mensual del BTC 
avg_month = df
avg_month["Month"] = [(mes[0:4] + "-" + mes[5:7]) for fecha in avg_month.index for mes in str(fecha).split()[0::2]]
avg_month_ = avg_month.groupby("Month").mean() 
display(avg_month_.reset_index(level=0))
        

,Avg yearly Bitcoin price (USD)
2011,5.642617
2012,8.289260
2013,189.057019
2014,526.924278
2015,272.178304
2016,567.468911
2017,3990.138147
2018,8324.522223


,Month,Bitcoin Price (USD)
0,2011-01,0.369174
1,2011-02,0.926393
2,2011-03,0.855987
3,2011-04,1.247870
4,2011-05,6.225158
5,2011-06,18.067877
6,2011-07,14.023535
7,2011-08,9.982903
8,2011-09,5.930853
9,2011-10,3.630900


## Web Scrapping

In [ ]:
url = ("https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html")
html = requests.get(url).content
result = BeautifulSoup(html,"lxml")


In [ ]:
country_ = result.select(".currency")
country = [ctry.text for ctry in country_]
currency_ = result.select(".alignLeft")
currency = [curr.text for curr in currency_]
spot_ = result.select(".rate")
spot = [sp.text for sp in spot_]
dictionary = {"Country":country,"Currency":currency,"Spot":spot}
df = pd.DataFrame(dictionary)
df

In [ ]:
A="A"
